In [14]:
import sys
sys.path.append("/home/julia/USF/spring2/productAnalytics/facenet/src")

from align import detect_face
from facenet import prewhiten, crop, flip, to_rgb

import tensorflow as tf
import numpy as np
import facenet
import os
import sys
import numpy as np
import pickle
import numpy as np
from scipy import misc
import cv2

In [18]:
def load_img(img, do_random_crop, do_random_flip, image_size, do_prewhiten=True):
    images = np.zeros((1, image_size, image_size, 3))
    if img.ndim == 2:
        img = to_rgb(img)
    if do_prewhiten:
        img = prewhiten(img)
    img = crop(img, do_random_crop, image_size)
    img = flip(img, do_random_flip)
    images[:,:,:,:] = img
    return images

In [9]:
def align_face(img, pnet, rnet, onet):

    minsize = 20 # minimum size of face
    threshold = [ 0.6, 0.7, 0.7 ]  # three steps's threshold
    factor = 0.709 # scale factor
    image_size = 160

    print("before img.size == 0")	
    if img.size == 0:
        print("empty array")
        return False,img,[0,0,0,0]

    if img.ndim<2:
        print('Unable to align')

    if img.ndim == 2:
        img = to_rgb(img)

    img = img[:,:,0:3]
    margin = 44

    bounding_boxes, _ = detect_face.detect_face(img, minsize, pnet, rnet, onet, threshold, factor)

    nrof_faces = bounding_boxes.shape[0]
    detect_multiple_faces = True
    
    if nrof_faces==0:
        return False,img,[0,0,0,0]
    else:
        det = bounding_boxes[:,0:4]
        det_arr = []
        img_size = np.asarray(img.shape)[0:2]
        if nrof_faces>1:
            if detect_multiple_faces:
                for i in range(nrof_faces):
                    det_arr.append(np.squeeze(det[i]))
            else:
                bounding_box_size = (det[:,2]-det[:,0])*(det[:,3]-det[:,1])
                img_center = img_size / 2
                offsets = np.vstack([ (det[:,0]+det[:,2])/2-img_center[1], (det[:,1]+det[:,3])/2-img_center[0] ])
                offset_dist_squared = np.sum(np.power(offsets,2.0),0)
                index = np.argmax(bounding_box_size-offset_dist_squared*2.0) # some extra weight on the centering
                det_arr.append(det[index,:])
        else:
            det_arr.append(np.squeeze(det))
        if len(det_arr)>0:
                faces = []
                bboxes = []
        for i, det in enumerate(det_arr):
            det = np.squeeze(det)
            bb = np.zeros(4, dtype=np.int32)
            bb[0] = np.maximum(det[0]-margin/2, 0)
            bb[1] = np.maximum(det[1]-margin/2, 0)
            bb[2] = np.minimum(det[2]+margin/2, img_size[1])
            bb[3] = np.minimum(det[3]+margin/2, img_size[0])
            cropped = img[bb[1]:bb[3],bb[0]:bb[2],:]
            scaled = misc.imresize(cropped, (image_size, image_size), interp='bilinear')
            faces.append(scaled)
            bboxes.append(bb)
            print("leaving align face")
        return True,faces,bboxes

from scipy.spatial.distance import cdist

def identify_person(image_vector, feature_names, feature_np, k=9):
    print(image_vector.shape)
    d = np.squeeze(cdist(image_vector, feature_np, metric='euclidean'))
    top_k_ind = np.argsort(d).tolist()[:k]
    result = feature_names[top_k_ind[0]]
    distance = d[top_k_ind[0]]
    result = result.split("_")[0]
    return result, distance

# def identify_person(image_vector, feature_dict, k=9):
#     top_k_ind = np.argsort([np.linalg.norm(image_vector-pred_row) \
#                         for ith_row, pred_row in enumerate(feature_dict.values())])[:k]
#     result = list(feature_array.keys())[top_k_ind[0]]
#     distance = np.linalg.norm(image_vector-list(feature_array.values())[top_k_ind[0]])
#     result = result.split("_")[0]
#     return result, distance

def recognize_face(sess,pnet, rnet, onet, feature_names, feature_np):
    images_placeholder = sess.graph.get_tensor_by_name("input:0")
    images_placeholder = tf.image.resize_images(images_placeholder,(160,160))
    embeddings = sess.graph.get_tensor_by_name("embeddings:0")
    phase_train_placeholder = sess.graph.get_tensor_by_name("phase_train:0")

    image_size = 160
    embedding_size = embeddings.get_shape()[1]

    cap = cv2.VideoCapture(-1)

    while(True):
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, 0)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            break
        if(gray.size > 0):
            print(gray.size)
            response, faces,bboxs = align_face(gray,pnet, rnet, onet)
            print(response)
            if (response == True):
                    for i, image in enumerate(faces):
                            bb = bboxs[i]
                            images = load_img(image, False, False, image_size)
                            feed_dict = { images_placeholder:images, phase_train_placeholder:False }
                            feature_vector = sess.run(embeddings, feed_dict=feed_dict)
                            result, distance = identify_person(feature_vector, feature_names, feature_np, 1)

                            if distance < 0.90:
                                print("result: {} acc: {}".format(result, distance))

                                cv2.rectangle(gray,(bb[0],bb[1]),(bb[2],bb[3]),(255,255,255),2)
                                W = int(bb[2]-bb[0])//2
                                H = int(bb[3]-bb[1])//2
                                cv2.putText(gray,result,(bb[0]+W-(W//2),bb[1]-7), cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1,cv2.LINE_AA)
                            else:
                                cv2.rectangle(gray,(bb[0],bb[1]),(bb[2],bb[3]),(255,255,255),2)
                                W = int(bb[2]-bb[0])//2
                                H = int(bb[3]-bb[1])//2
                                cv2.putText(gray,"",(bb[0]+W-(W//2),bb[1]-7), cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1,cv2.LINE_AA)
                            del feature_vector

            cv2.imshow('img',gray)
        else:
            continue

In [10]:
with open('extracted_dict.pickle','rb') as f:
    feature_dict = pickle.load(f)
    
feature_names = np.array(list(feature_dict.keys()))
feature_np = np.squeeze(list(feature_dict.values()))

model_exp = "20180408-102900/"
graph_fr = tf.Graph()
sess_fr = tf.Session(graph=graph_fr)

In [ ]:
with graph_fr.as_default():
    saverf = tf.train.import_meta_graph(os.path.join(model_exp, 'model-20180408-102900.meta'))
    saverf.restore(sess_fr, os.path.join(model_exp, 'model-20180408-102900.ckpt-90'))
    pnet, rnet, onet = detect_face.create_mtcnn(sess_fr, None)
    recognize_face(sess_fr, pnet, rnet, onet, feature_names, feature_np)